
# Homework 6 Processing Raw Text
Please send your solution as Jupyter notebooks (.ipynb) or Python files (.py). In case your submission consists of several files, compress these to a zip-file. Indicate clearly which submission corresponds to which question. Include comments in your program code to make it easier readable. Naming schema: YourName_Homework6.ipynb, YourName_Homework6.py or YourName_Homework6.zip. The deadline for the homework is Thursday, 02.12.2021 09:40 CET. Group submissions are not allowed.

In [139]:
import nltk
import pandas as pd
import re
from nltk.corpus import brown
from nltk.corpus.europarl_raw import english
from nltk.corpus import nps_chat
from nltk.corpus import webtext

nltk.download('europarl_raw')
nltk.download('gutenberg')
nltk.download('wordnet')

[nltk_data] Downloading package europarl_raw to
[nltk_data]     C:\Users\ZhuoY\AppData\Roaming\nltk_data...
[nltk_data]   Package europarl_raw is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\ZhuoY\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ZhuoY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
i=80
for i in webtext.words():
    print(i)

## Task 6.1 (2p)

### a. Create a function trigrams_regex(tokens), which finds from a given text all sequences of 3-grams that have an article, a potential adjective ending with -ing and a following word by using regular expressions only. 

Make sure that the third element of the 3-gram is a word (not punctuation). The function should take a list of tokens as an argument and return a list of all found n-grams. Try your function on austen-persuasion.txt from the Gutenberg corpus and print the output.

In [ ]:
def threeGrams(tokens:list): 
    '''
    To find all 3-grams which consists of an article, an adjective ending 
    with -ing and a following word.
    
    Only allow to match these grams by regular expresseion
    
    @param: a list of tokens
    @return: a list of all found n-grams.
    
    '''
    threegrams=[]
    tokens=iter(tokens)
    for token in tokens:
        if re.search(r'\bthe\b|\ban\b|\ba\b',token):
            adj=next(tokens)
            if re.search(r'^.*ing$',adj):
                word=next(tokens)
                if re.search(r"\.|-|!|\(|#|\$|&|\|%|\\|'|\)|\*|;|:|\?|,",word):
                    i=0
                else:
                    threegrams.append((token,adj,word))
    return threegrams
    
    
    
    
# show visual inspection
# TODO Duplicates?
text=nltk.corpus.gutenberg.words('austen-persuasion.txt')
df=pd.DataFrame()
df['3-grams']=threeGrams(text)
print(threeGrams(text))

### b. Considering the output of the previous task, describe the failure cases of the regular expression – cases where the -ing version is not an adjective. Think about how you can filter those whole matched word sequences containing an -ing word that is not an adjective. Adjust your function and run it one more time. Are there other cases that do not meet the condition “3-grams that have an article, an adjective ending with -ing and a following word”? How could you adjust the function by using other methods? Describe your solution in up to 4 sentences.

In [4]:
def threeGrams_2(tokens:list): 
    '''
    To find all 3-grams which consists of an article, an adjective ending 
    with -ing and a following word.
    
    Only allow to match these grams by regular expresseion
    
    **In contrast to the function above, it add a word list containing common 
    word which are ending with -ing but not adjective.**
    
    
    @param: a list of tokens
    @return: a list of all found n-grams.
    
    '''
    # a array consists of words with -ing suffix but are not adjective
    specialwords=['morning','beginning','evening','feeling','living','dancing','something','anything']
    threegrams=[]
    tokens=iter(tokens)
    for token in tokens:
        if re.search(r'\bthe\b|\ban\b|\ba\b',token):
            adj=next(tokens)
            if re.search(r'^.*ing$',adj) and adj not in specialwords:
                    word=next(tokens)
                    if re.search(r"\.|-|!|\(|#|\$|&|\|%|\\|'|\)|\*|;|:|\?|,",word):
                        i=0
                    else:
                        threegrams.append((token,adj,word))
    return threegrams

# show visual inspection
# TODO Duplicates?
text=nltk.corpus.gutenberg.words('austen-persuasion.txt')
df=pd.DataFrame()
df['3-grams']=threeGrams_2(text)
print(threeGrams_2(text))

[('the', 'blessing', 'of'), ('the', 'blessing', 'of'), ('the', 'neighbouring', 'market'), ('an', 'obliging', 'landlord'), ('a', 'throwing', 'away'), ('the', 'arranging', 'of'), ('the', 'preceding', 'month'), ('the', 'intervening', 'time'), ('a', 'projecting', 'tooth'), ('a', 'degrading', 'match'), ('the', 'making', 'of'), ('the', 'existing', 'connection'), ('the', 'declining', 'year'), ('the', 'meaning', 'of'), ('a', 'gleaning', 'of'), ('an', 'adjoining', 'hedge'), ('a', 'thing', 'that'), ('a', 'withdrawing', 'on'), ('a', 'relenting', 'on'), ('the', 'resembling', 'scenes'), ('the', 'preceding', 'summer'), ('a', 'pleasing', 'face'), ('a', 'thing', 'of'), ('a', 'bewitching', 'charm'), ('an', 'increasing', 'acquaintance'), ('the', 'sitting', 'down'), ('the', 'engaging', 'mildness'), ('the', 'flowing', 'of'), ('an', 'adjoining', 'apartment'), ('the', 'preceding', 'evening'), ('an', 'increasing', 'degree'), ('a', 'mortifying', 'reception'), ('the', 'governing', 'principle'), ('the', 'distre

Using `spaCy`

## Task 6.2 (2p)

Readability measures are used to score the reading difficulty of a text, for the purposes of selecting texts of appropriate difficulty for language learners. The Automated Readability Index (ARI) – a measure invented in
the 1960s and not in use anymore in state-of-the-art NLP – of a text is defined as:

$$
ARI=4.71\mu_{\omega}+0.5\mu_s-24.43.
$$

Where $\mu_\omega$ is the average number of letters per word, and $\mu_s$ is the average number of words per sentence in a given
text. As a rough guide, a score of 1 (is claimed to) correspond to the reading level at an age of 6 to 8, a score of 8
to the typical reading level of a 14 year-old US child. A score of 12 allegedly corresponds to the reading level of 17
years-old.

### a. Compute the ARI score for every category of the Brown Corpus. Which category is the easiest to understand? Which one is the most difficult one?


In [77]:
def ari_score(mo,ms):
    return 4.71*mo+0.5*ms-24.43

mu_omega=[]
mu_s=[]
for cate in brown.categories():
    totaletter=0
    for word in brown.words(categories=cate):
        totaletter+=len(word)
    mu_omega.append(totaletter/len(brown.words(categories=cate)))
    mu_s.append(len(brown.words(categories=cate))/len(brown.sents(categories=cate)))

df=pd.DataFrame(index=brown.categories())
df['mu_omega']=mu_omega
df['mu_s']=mu_s
df['ARI score']=df.apply(lambda x: ari_score(x['mu_omega'],x['mu_s']),axis=1)
df=df.sort_values('ARI score')
print('category '+df[:1].index.values[0]+'is the easiest to understand with ARI '+str(df.at[df[:1].index.values[0],'ARI score']))
print('category '+df[-1:].index.values[0]+' is the most difficult one with ARI '+str(df.at[df[-1:].index.values[0],'ARI score']))

category mysteryis the easiest to understand with ARI 0.8335518942055167
category government is the most difficult one with ARI 9.08430349501021


In [104]:
df.head()

,mu_omega,mu_s,ARI score
mystery,3.802078,14.711529,0.833552
adventure,3.829541,14.954065,1.084168
romance,3.795721,15.802753,1.349224
fiction,3.881351,16.118616,1.910474
science_fiction,3.986455,15.263713,1.978058


### b. (1 point) Use the europarl_raw.english corpus (Sample European Parliament Proceedings Parallel Corpus, we are only interested in the English texts) to prove the hypothesis: “Speeches are easier to understand than news.” 

Hint: you can use news from the Brown corpus .

In [102]:
def cal_ari(words:list,sents:list):
    '''
    @param corpus: 
    @return ari score
    '''
    totaletter=0
    for word in words:
        totaletter+=len(word)
    return ari_score(mo=totaletter/len(words),
                     ms=len(words)
                             /len(sents))
    
ari_news=cal_ari(brown.words(categories='news'),brown.sents(categories='news'))
ari_speech=cal_ari(english.words(),english.sents())
print("speech score: "+str(ari_speech))
print("news score: "+str(ari_news))

speech score: 10.744030503190167
news score: 7.176684595052684


## Task 6.3 (6 points) 

The T9 system1 is used for entering text on mobile phones (see the picture below). Two or more words that are entered with the same sequence of keystrokes are known as textonyms. For example, both “hole” and “golf” are entered by pressing the sequence “4653”. Implement an SMS decoder – a function that is similar to the T9 system on mobile phones and that translates from given digit sequences to words (e.g. “96753” -> “world”).

### a. (1 point) Choose at least one appropriate corpus and explain why you chose this corpus in one or two sentences. You will use the corpus to estimate which word is more frequent and should be a preferred output.

Using 'nps_chat' corpus. 
>The NPS Chat Corpus, Release 1.0 consists of 10,567 posts out of approximately 500,000 posts we have gathered from various online chat services in accordance with their terms of service. 
>- Part-of-speech tagged
>- Dialogue-act tagged

### b. (0.5 points) Define a reasonable mapping scheme for your decoder that maps characters to digits.

In [112]:
mapping=[
    (' '), # first element ([0]) stands for a whtiespace
    ('@','-','_'), # [1] represent number 1 on the T9 keyboard
    ('a','b','c'),
    ('d','e','f'),
    ('g','h','i'),
    ('j','k','l'),
    ('m','n','o'),
    ('p','q','r','s'),
    ('t','u','v'),
    ('w','x','y','z')
]

In [114]:
mapping={
    '2' : '[abc]',
    '3' : '[def]',
    '4' : '[ghi]',
    '5' : '[jkl]',
    '6' : '[mno]',
    '7' : '[pqrs]',
    '8' : '[tuv]',
    '9' : '[wxyz]'
}

### c. (3 points) Implement a function get_t9_word(digits,dictionary) which for a given sequence of digits,
e.g. “96753”, returns the most likely word, e.g. “world”. Multiple solutions are possible here. Your function might take another argument instead of a dictionary, but it should take the digits. You are also allowed to write and use additional functions.

In [178]:
%%time
def t9_word(digits:str,freq_dict):
    '''
    '''
    # generate regular expression. Each digital represents 3 possible alphabet
    rex='^'+''.join(mapping[dig] for dig in digits)+"\\b"
    match_word=''
    match_prob=0
    for k,v in freq_dict[len(digits)].items():
        if(re.search(rex,k,flags=re.IGNORECASE)):
            if v > match_prob: # select the matched word with highest occurence
                match_word=k
                match_prob=v
                
     # if no match return origin digits
    match_word=match_word if match_word else digits
    return match_word

def t9_wordlist(digitslist:list,freq_dict):
    words=list(t9_word(digits,freq_dict) for digits in digitslist)
    return words

corpus_words=nps_chat.words()+webtext.words()
freq=nltk.ConditionalFreqDist((len(word),word) for word in corpus_words)

test='43556'
print(t9_word(test,freq))

hello
Wall time: 4.13 s


### d. (1.5 points) Apply your decoder to the following digit sequences (the last two are sentences) and print the output:

In [167]:
# '273253278' =>breakfast

# '78674463' =>sunshine

# '53276' =>learn

# '33333'

# ['4663','6676464','469','927','9687','25277']

# ['273','968','3463']

Is the output been decoded in the right way?

In [179]:
%%time
print('"273253278" is '+ t9_word('273253278',freq))
print('"78674463" is '+ t9_word('78674463',freq))
print('"53276" is '+ t9_word('53276',freq))
print('"33333" is '+ t9_word('33333',freq))
print("['4663','6676464','469','927','9687','25277'] is ", 
      list(t9_wordlist(['4663','6676464','469','927','9687','25277'],freq)))
print("['273','968','3463'] is ", 
      list(t9_wordlist(['273','968','3463'],freq)))

"273253278" is breakfast
"78674463" is sunshine
"53276" is learn
"33333" is Eeeee
['4663','6676464','469','927','9687','25277'] is  ['good', 'morning', 'how', 'was', 'your', 'class']
['273','968','3463'] is  ['are', 'you', 'find']
Wall time: 68 ms


'3463' is decoded as "find" which is probabliy a wrong result. 'Are you fine' has more sense. The reason is that the occuence of "find" is more frequent than "fine". The *t9_word* function is not contet-aware or grammar-aware, it really depends on the frequence of the occuance of a word

In [188]:
print('The probability of "fine":'+str(freq[len('3463')].freq('fine')))
print('The probability of "fine":'+str(freq[len('3463')].freq('find')))

The probability of "fine":0.0019405172072393172
The probability of "fine":0.0027325650469288347


## Task 6.4 (not graded) 
This is an optional task that you might explore if you are interested. It will not count as a part of the official homework assignment. Improve the SMS decoder of homework 6.3 as follows:

(a) Take the context into account, guess the word using the bigram probability of the previous entered word with the
function get_t9_word(previous_word, digits). Test the improvement with the (context_word, digit) tuples
in the following list:

In [ ]:
print(get_t9_word('i', '26'))
print(get_t9_word('its', '26'))
print(get_t9_word('a', '3463'))
print(get_t9_word('will', '3463'))
print(get_t9_word('the', '22222'))

(b) Apply the decoder to each digit sequence in this “sentence”:
['43556','73837','4','26','3463']
Is the output readable? What errors have been made?